In [150]:
import numpy as np
import pandas as pd
import networkx as nx
import json
from collections import Counter
#nainštalujte pip install isbnlib
from isbnlib import meta
from isbnlib.registry import bibformatters

Pamäť pre celú sieť maticu susedností nedostaneme...:

In [174]:
# np.ones((10356390, 10356390))

MemoryError: Unable to allocate 780. TiB for an array with shape (10356390, 10356390) and data type float64

Dokonca ani pre malú časť vrcholov

In [173]:
# np.ones((87085, 87085))

MemoryError: Unable to allocate 56.5 GiB for an array with shape (87085, 87085) and data type float64

Vytvoríme si podgraf, vyberieme si náhodnú podmnožinu kníh (alebo čitateľov) a k nim všetky hrany (z druhej skupiny vrcholov zoberieme všetky potrebné vrcholy)

In [153]:
def subgraphByListOfValues(values: pd.Series, column: str, df: pd.DataFrame) -> pd.DataFrame:
    #
    return df[np.isin(df[column], values)]

In [154]:
def randomSubgraph(n:int, column: str, df: pd.DataFrame, weighed: bool = True, seed: int = 3, minDeg: int= 10) -> pd.DataFrame:
    np.random.seed(seed)
    values, counts = np.unique(df[column], return_counts=True)
    values, counts = values[counts > minDeg],  counts[counts > minDeg] # nech nemame knihy s velmi malo hodnoteniami (resp uzivatelov co majo citaju)
    return subgraphByListOfValues(np.random.choice(values, n, replace=False, p=(counts / sum(counts)) if weighed else None), column, df)

In [155]:
class RecommendationEngine:
    def __init__(self, filename: str, seed: int=3, n: int = 200, col: str = "ISBN"):
        self.df = pd.read_csv(filename, header=None, names=['ID_reviewer', 'ISBN','Stars','Date'])
        self.subdf = randomSubgraph(n, col, self.df, seed=seed)
        self.SM = {True: None, False: None}
        self.SMC = {True: None, False: None}
        self.G: nx.classes.graph.Graph = nx.from_pandas_edgelist(self.subdf, source='ID_reviewer', target='ISBN', edge_attr='Stars')
        self.userOrder = self.filterNodes(True)
        self.bookOrder = self.filterNodes(False)

    def filterNodes(self, users: bool) -> list[str]:
        # vypis zoznamu knih/citatelov
        return list(filter(lambda x: (x[0] == 'A') if users else (x[0] != 'A'), self.G.nodes()))

    def similarityMatrix(self, users: bool, save: bool=False) -> pd.DataFrame:  # use wisely!
        # vypocet matice podobnosti (poctu zhodnych susedov)
        if self.SM[users] is not None:
            return self.SM[users]
        userOrder = self.filterNodes(True)
        bookOrder = self.filterNodes(False)
        A = nx.algorithms.bipartite.matrix.biadjacency_matrix(self.G, userOrder, bookOrder).toarray()
        product = A @ A.T if users else A.T @ A
        np.fill_diagonal(product, 0) # citatel si so sebou nie je podobny (nech nam ho neodporuca)
        order = userOrder if users else bookOrder # priradenie id k hodnotam
        SM = pd.DataFrame(product, columns = order)
        SM.index = order
        if save:
            self.SM[users] = SM
        return SM

    def similarityMatrixCosine(self, users: bool, save: bool=False) -> np.ndarray:  # use wisely!
        # vypocet matice kosinusovej podobnosti
        if self.SMC[users] is not None:
            return self.SMC[users]
        SM = self.similarityMatrix(users)
        norms = (np.linalg.norm(SM, axis=1))
        SMC = SM / norms[:, None] / norms[None, :]
        np.nan_to_num(SMC, copy=False)
        if save:
            self.SMC[users] = SMC
        return SMC

    def similarityVectorForUser(self, user:str, userOrder:list[str] = None, cosine:bool = True):
        #vypocet vektora podobnosti usera so vsetkymi ostatnymi citatelmi
        if userOrder is None:
            userOrder = self.filterNodes(True)
        u = userOrder.index(user)
        A = nx.algorithms.bipartite.matrix.biadjacency_matrix(self.G, userOrder, weight='Stars').toarray()
        result = np.zeros((len(userOrder)))
        for r in range(len(userOrder)):
            result[r] = np.dot(A[r, :], A[u, :])
            if cosine:
                result[r] /= np.sqrt(np.dot(A[r, :], A[r, :]))
        if cosine:
            result /= np.sqrt(np.dot(A[u, :], A[u, :]))
        result[u] = 0
        return result, np.array(userOrder)

    def getNMostSimilarUsers(self, user:str, n:int, ignoreTotalMatch=True):
        # najdeme vektor podobnosti usera s vsetkymi ostatnymi
        sim, users = self.similarityVectorForUser(user, cosine=True)
        if ignoreTotalMatch:
            sim, users = sim[sim < 0.9999], users[sim < 0.9999] #odfiltrujeme dokonale zhody
        sim.argsort()
        return users[sim.argsort()[-n:]] # najdeme n najpodobnejsich

    def getSimilarBooksToABook(self, book):
        # vypocitame maticu podobnosti knih, najdeme a najpodobnejsiu knihu, zadanej knihe
        df = self.similarityMatrixCosine(False)
        return list(df[book][df[book] == df[book].max()].index)

    def getMetaFromISBN(self,isbn:str) -> json:
        try:
            SERVICE = "goob"
            format_json = bibformatters["json"]
            meta_data = (format_json(meta(isbn, SERVICE)))
            #print(meta_data)
            return meta_data
        except:
            return None

    def fromJsonToDict(self,json_file:json) -> dict:
        if json_file is None:
            return dict()
        return json.loads(json_file)

    def getDf(self)->pd.DataFrame:
      return self.subdf

    def getBooksByUser(self, id:str) ->list: #(list of ISBN for user)
      # df = df.loc[df['ID_reviewer']==id, 'ISBN'].values
      # return df
        return list(self.G.neighbors(id))

    def getBooks(self,df:pd.DataFrame,id1:str,id2:str,type_:str) ->list: #same - same books, diff - different books
      b1 = self.getBooksByUser(id1)
      b2 = self.getBooksByUser(id2)
      if type_ == "same":
        return list(set(b1).intersection(set(b2)))
      elif type_ == "diff1":
        return list(set(b2) - set(b1)) #vsetky knihy od id2, ktore id1 necital
      elif type_ == "diff2":
        return list(set(b1) - set(b2)) #vsetky knihy od id1, ktore id2 necital

    def getSimilarBooks(self,df:pd.DataFrame,id1:str,id2:str,type_:str) ->list:  #type ==(year, publisher)
        try:
            if type_ == "year": #odporucame id1 knihy od uzivatela id2 na zaklade roku vydania knihy
              books_s = self.getBooks(df,id1,id2,"same")
              #print(books_s)
              years = list()
              for i in range(len(books_s)): #zoberieme vseetky roky a pozrieme sa aky najviac prevlada -> zoberiem okolie (-5,5) a to odporucime
                pom = self.getMetaFromISBN(books_s[i])
                if pom != None: #nenaslo ISBN
                  years.append(self.fromJsonToDict(pom)["year"])
              most_common_era = Counter(years).most_common(1) #rok,pocet_opakovani
              most_common_era = [int(most_common_era[0][0])-5,int(most_common_era[0][0])+5]
              #print(most_common_era)
              books_d = self.getBooks(df,id1,id2,"diff1")
              recommendation_by_year = list()
              for i in range(len(books_d)): #zistime info o vsetkych kniha id2 a vyberieme nazvy tych ktorych vydania su medzi most_common_era
                pom = self.getMetaFromISBN(books_d[i])
                if pom != None: #nenaslo ISBN
                  if most_common_era[0] <= (int(self.fromJsonToDict(pom)["year"])) <= most_common_era[1]:
                    title = self.fromJsonToDict(pom)["title"]
                    year = self.fromJsonToDict(pom)["year"]
                    authors = list()
                    for i in range(len(self.fromJsonToDict(pom)["author"])):  #ak mame viac autorov
                      authors.append(self.fromJsonToDict(pom)["author"][i]['name'])
                    recommendation_by_year.append([title,year, authors])
                    #print(recommendation_by_year)
              if recommendation_by_year == []:
                recommendation_by_year = [("Nenasli sa ziadne knihy z rokov:",most_common_era[0],"-",most_common_era[1])]
              return recommendation_by_year

            elif type_ == "publisher": #odporucame id1 knihy od uzivatela id2, vyberame oblubeneho vydavatelstvo
               books_s = self.getBooks(df,id1,id2,"same")
               publishers = list()
               for i in range(len(books_s)): #zoberieme vseetky vydavatelstva -> most_common
                  pom = self.getMetaFromISBN(books_s[i])
                  if pom != None: #nenaslo ISBN
                    publishers.append(self.fromJsonToDict(pom)["publisher"])
               most_common_publisher = Counter(publishers).most_common(1) #vydavatelstvo,pocet_opakovani
               # print(most_common_publisher)
               books_d = self.getBooks(df,id1,id2,"diff1")
               recommendation_by_publisher = list()
               for i in range(len(books_d)): #zistime info o vsetkych kniha id2 a vyberieme nazvy tych ktorych vydania su od most_common_publishe
                  pom = self.getMetaFromISBN(books_d[i])
                  if pom != None: #nenaslo ISBN
                    if most_common_publisher[0] == (self.fromJsonToDict(pom)["publisher"]):
                      title = self.fromJsonToDict(pom)["title"]
                      year = self.fromJsonToDict(pom)["year"]
                      authors = list()
                      for i in range(len(self.fromJsonToDict(pom)["author"])): #ak mame viac autorov
                        authors.append(self.fromJsonToDict(pom)["author"][i]['name'])
                      recommendation_by_publisher.append([title,year, authors])
                    #print(recommendation_by_year)
               if recommendation_by_publisher == []:
                  recommendation_by_publisher = ["Nenasli sa ziadne knihy od oblubeneho vydavatela"]
               return recommendation_by_publisher
        except:
            return ["Nenasli sa ziadne knihy"]

In [156]:
re = RecommendationEngine('data/rec-amz-Books.edges', n=200, seed=76972)

In [157]:
print(f'Počet čitateľov: {len(re.filterNodes(True))}')

Počet čitateľov: 87085


O knihách vieme dohľadať info z datábazy:

In [158]:
meta_json_book = re.getMetaFromISBN("0061537934")
list_meta = re.fromJsonToDict(meta_json_book)
# print(list_meta)
print("ISBN: ", "0061537934")
print("Názov knihy: ",list_meta["title"])
print("Autor: ",list_meta["author"][0]['name'])
print("Rok vydania: ",list_meta["year"])
print("Vydavateľstvo: ",list_meta["publisher"])

ISBN:  0061537934
Názov knihy:  The Art Of Racing In The Rain
Autor:  Garth Stein
Rok vydania:  2008
Vydavateľstvo:  Harper


Ku knihám vieme odporučiť podobné

In [176]:
randomBook = np.random.choice(re.filterNodes(False))
simBs = re.getSimilarBooksToABook(randomBook)
simB = simBs[0]
print(f'most similar books to \"{re.fromJsonToDict(re.getMetaFromISBN(randomBook))["title"]}\"({randomBook}): \"{re.fromJsonToDict(re.getMetaFromISBN(simB))["title"]}\"({simB})')
print(f"similar books: {len(simBs)}") # vela podobnych znamena, ze kniha si nie je podobna so ziadnou knihou, teda je izolovana

most similar books to "Zoo"(0316097446): "The Tesla Gate"(1624671772)
similar books: 1


K používateľom vieme nájsť ďalších s podobným vkusom

In [160]:
randomUser = np.random.choice(re.filterNodes(True))
print(f'3 most similar users to {randomUser}: {re.getNMostSimilarUsers(randomUser, 3)}')

3 most similar users to A2BWZNAJCDKQ96: ['A1FTBXHM3DJTEA' 'ARSDJ482Z8MGV' 'AI3AUM07TTPE1']


A odporučiť ďalšie čítanie od čitateľov s podobným vkusom

In [171]:
user = np.random.choice(list(map(lambda x: x[0], filter(lambda x: x[1] > 5, list(re.G.degree(re.filterNodes(True)))))))
# a, b, c = re.getNMostSimilarUsers(user, 3)
recommendation = []
for u in re.getNMostSimilarUsers(user, 3):
    try:
        recommendation += re.getBooks(re.getDf(),user,u,"diff1")
    except:
        pass
recoStr = '\n'.join(map(lambda x: x['title'], filter(lambda x: "title" in x, map(lambda x: re.fromJsonToDict(re.getMetaFromISBN(x)), recommendation))))
already = re.getBooksByUser(user)
alrStr = '\n'.join(map(lambda x: x['title'], filter(lambda x: "title" in x, map(lambda x: re.fromJsonToDict(re.getMetaFromISBN(x)), already))))
print(f"Čitateľ {user} čítal \n{alrStr}\n Odporúčame mu knihy \n{recoStr}")
    # print(u)
    # books_users = re.getBooks(re.getDf(),user,u,"same")
    # print("same:" ,books_users)
    # books_users = re.getBooks(re.getDf(),user,u,"diff1")
    # print("diff1:" ,books_users)
    # books_users = re.getBooks(re.getDf(),user,u,"diff2")
    # print("diff2:" ,books_users)

Čitateľ AHUT55E980RDR čítal 
Shoot Don't Shoot
Blood Work
Lola's Secret - A Novel
S.E.C.R.E.T: S.E.C.R.E.T.--S.E.C.R.E.T.--1
Caring For A Loved One With Cancer
The Flamethrowers - A Novel
Exodus
 Odporúčame mu knihy 
The Atlantis Gene


Pre malú podsieť funkcie pre výber podľa vydavateľa alebo roku nefungujú dobre, potrebovali by sme väčšiu množinu kníh

In [184]:
print("Recommendation by year: ")
books_recomendation = re.getSimilarBooks(re.getDf(),"A2BWZNAJCDKQ96","A1FTBXHM3DJTEA","year")
print(books_recomendation)
print("Recommendation by publisher: ")
books_recomendation = re.getSimilarBooks(re.getDf(),"A2BWZNAJCDKQ96","A1FTBXHM3DJTEA","publisher")
print(books_recomendation)

Recommendation by year: 
['Nenasli sa ziadne knihy']
Recommendation by publisher: 
[]
['Nenasli sa ziadne knihy']
